# Generate daily volatility using dedicated service - F#

### Overview

This sample demonstrates how to request and plot from a dedicated data service **on-demand** `daily` volatility.

### Inputs/outputs
Volatility indicator requires instrument's identifier, date time intervals and time range as per inputs and returns the daily volatility.

N-points volatility is computed as follows:

$$ Volatility_N = \sqrt \frac{\sum_{i=1}^N(P_a - P_i)^2}{N}$$
Where:
* *$P_a$* is the average price of the *N prices*
* *$P_i$* is price i

This sample shows how to plot a simple daily vwap for basis technical analysis using an open source library. 

### Services used
This sample uses gRPC requests in order to retrieve daily volatility indicator from the hosted service. The queried endpoint in this script are:
* *DailyVolatilityService *: to directly retrieve daily volatility objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.DailyAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run daily VWAP sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.9.*-pre*"

In [ ]:
open Systemathics.Apis.Helpers
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.DailyAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
let headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
let channel = ChannelHelpers.GetChannel();

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily volatility* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates

In [ ]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

In [ ]:
// Set data interval (we are using Google date time format)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = 2019, Month = 01, Day = 01 ),
    EndDate = new Date ( Year = 2020, Month = 12, Day = 31 )
  )

// Generate constraints based on the previous date selection
// Time constraints are not neeeded for the splits
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Instantiate the daily volatility service
let service = new DailyVolatilityService.DailyVolatilityServiceClient(channel)

// Create and process daily volatility request
let request = 
    new DailyVolatilityRequest( 
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker),
        Constraints = constraints
    )
let reply = service.DailyVolatility(request, headers)

The following code snippet displays the *daily bars* data points:

In [ ]:
reply.Value |> display